# **네이버 영화 리뷰 감정 분석 with BERT** 
BERT(Bidirectional Encoder Representations from Transformers)는 구글이 개발한 사전훈련(pre-training) 모델입니다. 학습된 언어 모델을 이용하여 실제 자연어 처리 문제를 푸는 파인 튜닝을 수행하여, 네이버 영화리뷰 감정분석을 수행하려 합니다.
사전 훈련된 모델을 사용하면 보다 적은 데이터를 가지고 보다 더 빠르게 학습할 수 있다는 장점이 있습니다. 또한 BERT 언어 모델에 CNN이나 LSTM, ATTENTION과 같은 복잡한 기법을 사용하지 않고 DNN만으로도 꽤 좋은 성능을 보장한다는 장점이 있습니다. CNN이나 LSTM등을 사용해도 크게 성능 개선이 되지 않아서 본 프로젝트 발표에서는 DNN만 간단하게 추가해서 작업해보았습니다. PyTorch를 이용하여 수행하신 분들의 코드를 쉽게 찾을 수 있는데, PyTorch를 사용하는 방법을 알지 못해 다르게 코드로 작업해보았습니다. 프로젝트 말미에 레퍼런스들을 남겨두었으니 참고하시면 좋을 것 같습니다.

## 라이브러리

분석에 필요한 라이브러리를 설치해줍니다. 모델간 충돌 혹은 에러 발생 가능성을 줄이기 위해 transformers와 tensorflow를 제시된 버전으로 설치합니다.

In [1]:
!pip install transformers==2.11.0
!pip install tensorflow==2.2.0

     |████████████████████████████████| 674 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 85.0 MB/s 
     |████████████████████████████████| 5.6 MB 61.0 MB/s 
     |████████████████████████████████| 1.2 MB 82.7 MB/s 
     |████████████████████████████████| 516.2 MB 3.5 kB/s 
     |████████████████████████████████| 2.9 MB 68.5 MB/s 
     |████████████████████████████████| 3.0 MB 62.2 MB/s 
     |████████████████████████████████| 454 kB 64.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import os
import re
import copy
import json
import urllib.request

from tqdm import tqdm
from transformers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## 데이터 다운로드 & EDA

박은정님의 네이버 영화리뷰 감정분석 데이터를 urllib을 이용해 불러옵니다.

In [3]:
train_file = urllib.request.urlopen("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt")
test_file = urllib.request.urlopen("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt")

train_data = pd.read_table(train_file)
test_data = pd.read_table(test_file)

데이터를 정제하는 과정에서, 실제 감정 분석에 쓰일 텍스트에 관한 처리를 수행하였습니다.

중복되는 값들을 없애주었고, 실제로 감정에 영향을 미치지 못하는 특수문자들을 제거하였습니다. 

그리고 데이터가 존재하지 않는 부분들도 모두 삭제하였습니다.

In [4]:
train_data.drop_duplicates(subset=['document'], inplace=True)
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['document'] = train_data['document'].str.replace('^ +', "")
train_data['document'].replace('', np.nan, inplace=True)

In [5]:
train_data.isnull().sum()

id            0
document    790
label         0
dtype: int64

In [6]:
train_data.dropna(inplace = True)

In [29]:
test_data.dropna(inplace = True)

데이터 전처리가 끝난 훈련 데이터와 테스트 데이터의 일부분을 살펴보시면 전처리 작업이 잘 수행된 것을 알 수 있습니다.
그리고 데이터의 라벨이 0이면 부정적인 감정을 담고 있고, 라벨이 1이면 긍정적인 감정을 담고 있다고 분류됩니다.

In [18]:
test_data.dtypes

id           int64
document    object
label        int64
dtype: object

In [19]:
train_data.dtypes

id           int64
document    object
label        int64
dtype: object

In [7]:
train_data.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [8]:
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


## BertTokenizer

제가 작업한 것들을 재현할 수 있도록, Seed 값을 설정해줍니다.
추후에 사용할 하이퍼 파라미터 값들도 지정해주도록 하겠습니다.

In [9]:
tf.random.set_seed(77)
np.random.seed(77)

BATCH_SIZE = 32
MAX_LEN = 64
NUM_EPOCHS = 4
VALID_SPLIT = 0.2

토크나이저는 여러 언어의 데이터를 기반으로 만든 'bert-base-multilingual-cased'를 사용해 한국어에도 적용할 수 있습니다. 

한국어는 소문자가 없기 때문에 do_lower_case를 False로 지정해줍니다.

토크나이저를 할 수 있는 함수를 여러 코드들을 참조하여 만들었습니다.

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', cache_dir='bert_ckpt', do_lower_case=False)

def bert_tokenizer(sentence, MAX_LEN):
    encoded_dict = tokenizer.encode_plus(
        text = sentence,
        add_special_tokens = True,
        max_length = MAX_LEN,
        pad_to_max_length = True,
        return_attention_mask = True
    )

    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']

    return input_id, attention_mask, token_type_id

빈 리스트를 만들고, 반복문을 작성합니다. bert_tokenizer함수에 train_setence를 집어 넣어, Bert모델에 필요한 파라미터들의 값을 채워넣습니다.

sentence의 수와 label의 수가 각각 정상적으로 나왔습니다.

In [11]:
input_ids = []
attention_masks = []
token_type_ids = []
train_data_labels = []

for train_sentence, train_label in tqdm(zip(train_data['document'], train_data['label']), total=len(train_data)):
    input_id, attention_mask, token_type_id = bert_tokenizer(train_sentence, MAX_LEN)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    train_data_labels.append(train_label)

train_movie_input_ids = np.array(input_ids, dtype=int)
train_movie_attention_masks = np.array(attention_masks, dtype=int)
train_movie_token_type_ids = np.array(token_type_ids, dtype=int)

train_movie_inputs = (train_movie_input_ids, train_movie_attention_masks, train_movie_token_type_ids)
train_data_labels = np.asarray(train_data_labels, dtype=np.int32)

print("Sentences: {}\n Labels: {}".format(len(train_movie_input_ids), len(train_data_labels)))

100%|██████████| 145393/145393 [00:33<00:00, 4331.59it/s]


Sentences: 145393
 Labels: 145393


특정 인덱스의 값을 조회하여 작업이 정상적으로 잘 수행되었는지 확인합니다.

가장 직관적으로 확인할 수 있는 tokenizer.decode(input_id)를 확인하시면, CLS와 PAD로 잘 구성되어있다는 것을 확인하실 수 있습니다.

In [12]:
idx = 77

input_id = train_movie_input_ids[idx]
attention_mask = train_movie_attention_masks[idx]
token_type_id = train_movie_token_type_ids[idx]

print(input_id)
print(attention_mask)
print(token_type_id)
print(tokenizer.decode(input_id))

[  101 42608  9489 89292   102     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[CLS] 매우 실망 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

작성된 모델을 살펴보시면, dropout과 간단한 DNN만을 추가하였다는 것을 보실 수 있을 겁니다.

긍정 부정의 분류이기 때문에 class의 수가 2로 설정된 것도 확인하실 수 있습니다.

In [13]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class,
                                                kernel_initializer = tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range),
                                                name='classifier')

    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

cls_model = TFBertClassifier(model_name = 'bert-base-multilingual-cased',
                             dir_path = 'bert_ckpt',
                             num_class = 2)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

## 모델 학습

optimizer의 학습률도 설정해주고, loss함수를 sparse_categorical_crossentropy로 설정합니다. 

메모리와 시간을 아낄 수 있고, 샘플들이 정확히 하나의 클래스에 속하는 경우 좋다고 하여 이 함수로 정했습니다.

그 후 accuracy로 설정하고 컴파일 하였습니다.


In [14]:
optimizer = tf.keras.optimizers.Adam(3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

긴 작업 시간 중 사고를 대비하여 체크포인트를 설정하였습니다. 

12시간의 작업이 사라지고 난 후에 찾아서 작성하였습니다. 

model.fit 과정에서는 앞서 설정해두었던, 하이퍼 파라미터들을 이용해서 작성하였습니다. 

In [15]:
model_name = 'tf2_bert_naver_movie'

es_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)

checkpoint_path = os.path.join('./', model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
    print("{} Dir already exist\n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} Create Dir Completely\n".format(checkpoint_dir))

cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy',
                              verbose=1, save_best_only=True, save_weights_only=True)

history = cls_model.fit(train_movie_inputs, train_data_labels,
                        epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_split=VALID_SPLIT,
                        callbacks=[es_callback, cp_callback])

./tf2_bert_naver_movie Create Dir Completely

Epoch 1/4
3635/3635 [==============================] - ETA: 0s - loss: 0.4199 - accuracy: 0.8015
Epoch 00001: val_accuracy improved from -inf to 0.84074, saving model to ./tf2_bert_naver_movie/weights.h5
3635/3635 [==============================] - 1016s 280ms/step - loss: 0.4199 - accuracy: 0.8015 - val_loss: 0.3684 - val_accuracy: 0.8407
Epoch 2/4
3635/3635 [==============================] - ETA: 0s - loss: 0.3288 - accuracy: 0.8556
Epoch 00002: val_accuracy improved from 0.84074 to 0.85192, saving model to ./tf2_bert_naver_movie/weights.h5
3635/3635 [==============================] - 1014s 279ms/step - loss: 0.3288 - accuracy: 0.8556 - val_loss: 0.3370 - val_accuracy: 0.8519
Epoch 3/4
3635/3635 [==============================] - ETA: 0s - loss: 0.2742 - accuracy: 0.8830
Epoch 00003: val_accuracy did not improve from 0.85192
3635/3635 [==============================] - 1011s 278ms/step - loss: 0.2742 - accuracy: 0.8830 - val_loss: 0.3580 

다음과 같이 간단한 형태로도, 괜찮은 성능의 모델이 만들어 진 것으로 보입니다.

In [16]:
cls_model.summary()

Model: "tf_bert_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_bert_model (TFBertModel)  multiple                  177853440 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 177,854,978
Trainable params: 177,854,978
Non-trainable params: 0
_________________________________________________________________


## 모델 평가

아까 훈련 데이터를 가지고 했던 작업을 테스트 데이터에도 적용하여, 모델을 평가합니다.

In [32]:
input_ids = []
attention_masks = []
token_type_ids = []
test_data_labels = []

for test_sentence, test_label in tqdm(zip(test_data['document'], test_data['label'])):
    input_id, attention_mask, token_type_id = bert_tokenizer(test_sentence, MAX_LEN)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    test_data_labels.append(test_label)

test_movie_input_ids = np.array(input_ids, dtype=int)
test_movie_attention_masks = np.array(attention_masks, dtype=int)
test_movie_token_type_ids = np.array(token_type_ids, dtype=int)

test_movie_inputs = (test_movie_input_ids, test_movie_attention_masks, test_movie_token_type_ids)
test_data_labels = np.asarray(test_data_labels, dtype=np.int32)

48389it [00:11, 4201.56it/s]


In [33]:
cls_model.evaluate(test_movie_inputs, test_data_labels, batch_size = 512)

95/95 [==============================] - 97s 1s/step - loss: 0.3843 - accuracy: 0.8482


[0.3843192160129547, 0.8482093214988708]

# Reference

https://stackoverflow.com/questions/61708486/whats-difference-between-tokenizer-encode-and-tokenizer-encode-plus-in-hugging

http://yonghee.io/bert_binary_classification_naver/

https://ebbnflow.tistory.com/151

https://github.com/NLP-kr/tensorflow-ml-nlp-tf2

https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=encode_plus#transformers.PreTrainedTokenizer.encode_plus